In [1]:
import boto3, re
from sagemaker import get_execution_role
role = get_execution_role()

In [2]:
!cat entry_point.py



# Import to list directories 
import os

print(os.getcwd())
print(os.listdir(os.getcwd()))

### import for time and time monitoring 
import time
## Imports for Metrics operations 
import numpy as np 
## Imports for dataframes and .csv managment
import pandas as pd 

# TensorFlow library  Imports
import tensorflow as tf
print("tf version: ", tf.__version__)



import swish_package
from swish_package import swish



# Keras (backended with Tensorflow) Imports
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import CSVLogger

# Sklearn package for machine learining models 
## WILL BE USED TO SPLIT  TRAIN_VAL DATASETS
from sklearn import metrics
from sklearn.model_selection import train_te

In [ ]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('1', recursive=True)


In [3]:
import sagemaker
sagemaker_session = sagemaker.Session()
#inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='modell')

In [4]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/modell/model.tar.gz',
                                  role = role,
                                  framework_version = '2.3',
                                  entry_point = 'inference.py',
                                  source_dir='my_src',
                                  env={'SAGEMAKER_REQUIREMENTS': 'requirements.txt'}                                           
                                 )

In [5]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1 , instance_type='ml.m5.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


------!CPU times: user 1min 28s, sys: 12.6 s, total: 1min 41s
Wall time: 4min 53s


In [ ]:
predictor.endpoint

In [51]:
!pip install transformers==2.10

     |████████████████████████████████| 660 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 20.2 MB/s eta 0:00:01
     |████████████████████████████████| 723 kB 63.4 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 69.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 65.2 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=58ce18c4490a1dd9c04383d1d94f66e079945ab3594c699deeeae94b3fe56b04
  Stored in directory: /home/ec2-user/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2
Successfully built sacremoses
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


In [24]:
import os 

import transformers
from transformers.modeling_tf_distilbert import TFDistilBertModel
# TensorFlow library  Imports
import tensorflow as tf
# Keras (backended with Tensorflow) Imports
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import CSVLogger

In [52]:
from transformers import DistilBertTokenizer
# Configuration
MAX_LEN = 192
#MODEL = 'jplu/tf-xlm-roberta-large'
#MODEL = 'roberta-base'
MODEL = 'distilbert-base-multilingual-cased'
EPOCHS = 10
tokenizer = DistilBertTokenizer.from_pretrained(MODEL)


In [41]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    return np.array(enc_di['input_ids'])

In [42]:
import numpy as np 
D = np.array(["I want to see you"])

In [43]:
x_test = regular_encode(D, tokenizer, maxlen=MAX_LEN)

In [45]:
x_test

array([[  101,   146, 21528, 10114, 12888, 13028, 13935,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [57]:
x = "Tu devrais mourir bientôt"
pred = predictor.predict(x)

In [58]:
pred

{'predictions': [[0.953224897]]}

In [8]:
round(pred['predictions'][0][0])

1